In [ ]:
import numpy as np
import scipy
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from math import ceil
import os

In [ ]:
nTurn = 128
pipe_radius = 0.02*2.0**0.5

In [ ]:
beam,lattice = impact.readInputFile('test.SF.in')
ke   = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

lattice=[lattice[0],lattice[2]]

betx = lattice[1].betx
NL_t = lattice[1].strength_t
NL_c = lattice[1].transverse_scale_c
lattice[1].n_sckick = 50
lattice[1].n_map = 5

### Check pData

In [ ]:
pData = impact.readParticleData('partcl.data.chad',ke,mass,freq)
plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.scatter(pData[:,0],pData[:,2],s=0.1)
plt.subplot(2,2,2)
plt.scatter(pData[:,1],pData[:,3],s=0.1)
plt.subplot(2,2,3)
plt.scatter(pData[:,0],pData[:,1],s=0.1)
plt.subplot(2,2,4)
plt.scatter(pData[:,2],pData[:,3],s=0.1)
plt.tight_layout()

### gen Test pData

In [ ]:
xmax  = np.max(pData[:,0])
pxmax = np.max(pData[:,1])
ymax  = np.max(pData[:,2])
pymax = np.max(pData[:,3])

ny = 100
nx = int(xmax/ymax*ny)
npx = 100
npy = int(pymax/pxmax*npx)

q_m = beam.multi_charge.q_m[0]
k=0
pTest = []
for x in np.linspace(-xmax,xmax,nx):
    for y in np.linspace(-ymax,ymax,ny):
        for px in np.linspace(-pxmax,pxmax,npx):
            for py in np.linspace(-pymax,pymax,npy):
                k+=1
                pTest.append([x,px,y,py,0.0,0.0,q_m,0.0,k])
pTest = np.array(pTest)

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.scatter(pTest[:,0],pTest[:,2],s=0.1)
plt.subplot(2,2,2)
plt.scatter(pTest[:,1],pTest[:,3],s=0.1)
plt.subplot(2,2,3)
plt.scatter(pTest[:,0],pTest[:,1],s=0.1)
plt.subplot(2,2,4)
plt.scatter(pTest[:,2],pTest[:,3],s=0.1)
plt.tight_layout()

In [ ]:
dist = scipy.stats.gaussian_kde(pData[:,:4].transpose())

In [ ]:
weight = dist(pTest[:,:4].T)
plt.hist(weight)

In [ ]:
dist.pdf([0.007,0.0,0.0,0.0])

In [ ]:
dist([0.007,0.0,0.0,0.0])

In [ ]:
dist([0.007,0.0,0.0007,0.0])

In [ ]:
iCore = np.argwhere(weight > 1.0e6) # empirically found ...

pTestCore = pTest[iCore,:]

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.scatter(pData[:,0],pData[:,2],s=0.1)
plt.scatter(pTestCore[:,0],pTestCore[:,2],s=0.4)
plt.subplot(2,2,2)
plt.scatter(pData[:,1],pData[:,3],s=0.1)
plt.scatter(pTestCore[:,1],pTestCore[:,3],s=0.4)
plt.subplot(2,2,3)
plt.scatter(pData[:,0],pData[:,1],s=0.1)
plt.scatter(pTestCore[:,0],pTestCore[:,1],s=0.4)
plt.subplot(2,2,4)
plt.scatter(pData[:,2],pData[:,3],s=0.1)
plt.scatter(pTestCore[:,2],pTestCore[:,3],s=0.4)
plt.tight_layout()

print(len(pTestCore))
# pData = None
# dist = None
# weight = None

In [ ]:
pTestCore = pTest[iCore,:]
print(len(pTestCore))
print(len(pTest))
print(np.sum(iCore))

In [ ]:
weight

In [ ]:
latticeTmp = [impact.getElem('loop'),impact.getElem('write_raw_ptcl')]
latticeTmp[0].turns=1
latticeTmp[-1].file_id = -111110
latticeTmp[-1].turn = 1
beam.n_particles = nptTest
impact.writeParticleData(pTest, ke, mass, freq)
beam.nCore_y = 1
beam.nCore_z = 1
impact.writeInputFile(beam,latticeTmp)
impact.run(beam)

In [ ]:
pData = impact.readParticleData(-111110,ke,mass,freq)
plt.figure(figsize=(3,3))
plt.scatter(pData[:,0],pData[:,2],s=0.01)

In [ ]:
TBT = impact.getElem('TBT')
TBT.file_id = nTurn
lattice.insert(1,TBT)

TBT_integral = impact.getElem('TBT_integral')
TBT_integral.file_id = nTurn
TBT_integral.betx = betx
TBT_integral.alfx = 0.0
TBT_integral.strength_t = NL_t
TBT_integral.transverse_scale_c = NL_c
lattice.insert(2,TBT_integral)

if nTurn == 128:
    lattice[0].turns = nTurn
else:
    lattice[0].turns = int(nTurn/2)

for i in range(len(lattice)):
    if 'pipe_radius' in lattice[i].keys() and lattice[i].type!='dipole':
        lattice[i].pipe_radius = pipe_radius
        
lattice.append(impact.getElem('write_raw_ptcl'))
if nTurn == 128:
    lattice[-1].turn = nTurn
else:
    lattice[-1].turn = int(nTurn/2)

lattice[-1].file_id = -nTurn

In [ ]:
beam.distribution.distribution_type = 'ReadFile_binary'
if nTurn == 128:
    beam.distribution.file_id = 111110
else:
    beam.distribution.file_id = int(nTurn/2)
beam.nCore_y = 6
beam.nCore_z = 4
impact.writeInputFile(beam,lattice,'test.in.'+str(nTurn)+'.forward') 
#impact.run(beam)

In [ ]:
with open('test.in.'+str(nTurn)+'.forward') as f:
    testIn=f.read()
print(testIn)